In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from sklearn import tree
from sklearn.externals.six import StringIO

import os

In [2]:
def getPageSource(url):
    if os.name=='nt':
        driver=webdriver.PhantomJS(executable_path='../phantomjs.exe')
    else:
        driver=webdriver.PhantomJS(executable_path='../phantomjs')
    # kill phantomJS
    if os.name=='nt':
        os.system('taskkill /f /im phantomjs.exe')
    
    driver.get(url)
    html=driver.page_source
    return html

In [3]:
def getSerial(year,month):
    url='http://score.sports.media.daum.net/schedule/baseball/kbo/main.daum?game_year=%04d&game_month=%02d'%(year,month)
    html=getPageSource(url)
    html=BeautifulSoup(html)
    serial=html.select('a.btn_comm.btn_result, a.btn_comm.btn_highlight')
    holder='http://sports.media.daum.net/sports/gamecenter/'
    result=[]
    
    for element in serial:
        s=element.attrs['href']
        result.append(s[len(holder):len(holder)+8])
    return result

In [4]:
year_list=range(2015,2017)
month_list=range(3,10)
serial_list=[]
for year in year_list:
    if year==2016:
        month_list=range(3,8)
    for month in month_list:
        serial_list.append(getSerial(year,month))

/home/gohyunyoung98/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [5]:
serial=[x for  element in serial_list for x in element]

In [8]:
# score_gap
f=open('extractCrticalInning.csv','w')
for element in serial:
   
    url = str.format('http://m.sports.media.daum.net/m/sports/pack/3min/%s?result'%(element))
    html=getPageSource(url)
    html=BeautifulSoup(html)
    
    away_score=[int(element.text) for element in html.select('tr.away div.td_point') if element.text != '' and element.text!='-']
    # fill zero
    while(len(away_score)<12):
        away_score.append(0)
    # append sum_score
    away_score.append(int(html.select_one('tr.away td.run').text))
    
    home_score=[int(element.text) for element in html.select('tr.home div.td_point') if element.text!='' and element.text!='-']
    # fill zero
    while(len(home_score)<12):
        home_score.append(0)
    # append sum_score
    home_score.append(int(html.select_one('tr.home td.run').text))
    
    score_gap=[x-y for x,y in zip(away_score,home_score)]
    
    critical_inning_tuple=html.select('tbody strong.img_highlight')
    
    critical_inning=[int(x.parent.parent.parent['data-inning']) for x in critical_inning_tuple]
    
    # more than 1 critical_inning
    
    for inn in critical_inning:
        score_gap.append(inn)
        f.write('%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d\n'%tuple(score_gap))
        print('%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d'%tuple(score_gap))
        score_gap.pop()
f.close()

0,-1,0,0,-1,0,0,1,0,0,0,0,-1,8
0,-1,0,0,-1,0,0,1,0,0,0,0,-1,8
0,1,0,0,0,-1,0,0,1,0,0,0,1,5
0,1,0,0,0,-1,0,0,1,0,0,0,1,8
0,1,-5,0,-1,-1,0,-2,0,0,0,0,-8,8
0,1,-5,0,-1,-1,0,-2,0,0,0,0,-8,3
0,1,-5,0,-1,-1,0,-2,0,0,0,0,-8,5
0,1,-5,0,-1,-1,0,-2,0,0,0,0,-8,8
1,0,0,0,0,0,3,0,0,0,0,0,4,8
0,0,0,0,0,0,0,0,-1,0,0,0,-1,8
0,0,0,0,-3,-2,0,0,0,0,0,0,-5,8
-1,-2,-3,0,3,-3,0,0,0,0,0,0,-6,8
0,-2,0,2,1,1,4,-1,0,0,0,0,5,8
0,-2,0,2,1,1,4,-1,0,0,0,0,5,8
0,-4,-2,0,0,0,0,-3,0,0,0,0,-9,5
0,-4,-2,0,0,0,0,-3,0,0,0,0,-9,8
0,0,-1,2,0,0,0,0,0,0,0,0,1,1
-2,0,0,1,-5,0,-2,2,0,0,0,0,-6,5
-2,0,0,1,-5,0,-2,2,0,0,0,0,-6,8
0,2,0,5,0,-3,1,0,-1,0,0,0,4,5
0,2,0,5,0,-3,1,0,-1,0,0,0,4,8
0,0,0,0,0,0,0,2,0,0,0,0,2,5
0,0,0,0,0,0,0,2,0,0,0,0,2,8
-2,2,-1,0,5,0,2,-1,1,0,0,0,6,5
-3,0,1,0,-5,2,-2,0,0,0,0,0,-7,5
0,3,-1,1,0,0,4,0,3,0,0,0,10,4
0,0,1,0,1,-1,-1,2,0,0,0,0,2,4
0,0,1,0,1,-1,-1,2,0,0,0,0,2,9
-2,0,-1,1,-1,1,0,-1,1,0,0,0,-2,8
-1,0,1,0,0,0,0,0,0,0,0,0,0,3
-1,0,1,0,0,0,0,0,0,0,0,0,0,5
-1,0,1,0,0,0,0,0,0,0,0,0,0,5
4,0,0,0,0,5,0,0,3,0,

AttributeError: 'NoneType' object has no attribute 'text'

In [173]:
# DecisionTree - 서두
path_INTRO = './DecisionLearning//DecisionTree_sample.csv'
df_INTRO = pd.read_csv(path_INTRO,index_col=None,header=None,names=['1H','2H','3H','4H','5H','6H','7H','8H','9H','10H','11H','12H','sum','context'])
score_INTRO = np.array(df_INTRO.reindex(columns=['1H','2H','3H','4H','5H','6H','7H','8H','9H','10H','11H','12H','sum']))
context_INTRO = np.array(df_INTRO.get('context'))
context_sen_INTRO = [' ','승리하였다. ','접전끝에 승리를 하였다. ','역전에 성공했다. ','완승을 하였다. ','무승부로 끝이났다. ','영봉승을 하였다. ']
clf_INTRO = tree.DecisionTreeClassifier(criterion='entropy')
clf_INTRO = clf_INTRO.fit(score_INTRO,context_INTRO)


<class 'sklearn.datasets.base.Bunch'>
